### Instrucciones para Descargar los Datos desde Kaggle

Para descargar los archivos de datos de la competencia de Kaggle directamente a este entorno de Google Colab, sigue los siguientes pasos:

1. **Generar el archivo `kaggle.json`**:
   - Ve a [Kaggle](https://www.kaggle.com).
   - Inicia sesión con tu cuenta.
   - Dirígete a tu perfil (haciendo clic en tu imagen de perfil en la esquina superior derecha) y selecciona **Settings**.
   - Desplázate hasta la sección **API** y haz clic en **Create New API Token**.
   - Esto descargará un archivo `kaggle.json` en tu computadora. Este archivo contiene las credenciales necesarias para acceder a la API de Kaggle.

2. **Subir el archivo `kaggle.json` a Google Colab**:
   - Ejecuta la celda que te pide subir archivos.
   - Selecciona el archivo `kaggle.json` que descargaste de Kaggle.

3. **Ejecutar el código para descargar los datos**:
   - Una vez que hayas subido el archivo `kaggle.json`, las siguientes celdas descargarán y descomprimirán automáticamente los archivos de la competencia desde Kaggle.

**Subir el archivo `kaggle.json`**:

In [ ]:
!pip install kaggle

# Solicitar el archivo kaggle.json
from google.colab import files
files.upload()  # Subir el archivo kaggle.json

# Crear el directorio .kaggle y mover el archivo kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


**Código para descargar los datos**:


In [ ]:
# Descargar los datos
!kaggle competitions download -c udea-ai4eng-20242

# Descomprimir los archivos descargados
!unzip udea-ai4eng-20242.zip

 99% 20.0M/20.1M [00:02<00:00, 12.8MB/s]
100% 20.1M/20.1M [00:02<00:00, 9.05MB/s]
Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


**Importando librerías**

In [ ]:
# Importar librerías
!pip install optuna
!pip install category_encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import f_oneway
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 10.2 MB/s eta 0:00:00


# **Carga y visualización del dataset**

In [ ]:
# Cargar los datos
data = pd.read_csv('train.csv')

# Mostrar los primeros registros
data.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


**Información del dataset**

In [ ]:
# Mostrar tipos de datos y valores nulos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_EDUCACIONMADRE             668836 non-null  object
 10  ESTU_PAGOMATRICULAPROPIO        686002 non-null  object
 11  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(2), object(10)
memory

**Identificar valores faltantes**

In [ ]:
# Explorar la cantidad de valores faltantes por columna
missing_values = data.isnull().sum()
print("Valores faltantes:\n", missing_values)

Valores faltantes:
 ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_EDUCACIONMADRE               23664
ESTU_PAGOMATRICULAPROPIO           6498
RENDIMIENTO_GLOBAL                    0
dtype: int64


#**Limpieza y preprocesamiento de datos**

In [ ]:
# Crear una copia del dataset original
data_cleaned = data.copy()

# Eliminar la columna ID del dataset
if 'ID' in data_cleaned.columns:
    data_cleaned = data_cleaned.drop(columns=['ID'])
    print("Columna 'ID' eliminada correctamente.")
else:
    print("La columna 'ID' ya fue eliminada.")

Columna 'ID' eliminada correctamente.


**Porcentaje de valores faltantes**

In [ ]:
# Calcular el porcentaje de valores faltantes por columna
missing_percentage = data_cleaned.isnull().sum() / len(data_cleaned) * 100
print(missing_percentage)

PERIODO                           0.000000
ESTU_PRGM_ACADEMICO               0.000000
ESTU_PRGM_DEPARTAMENTO            0.000000
ESTU_VALORMATRICULAUNIVERSIDAD    0.907870
ESTU_HORASSEMANATRABAJA           4.455884
FAMI_ESTRATOVIVIENDA              4.640722
FAMI_TIENEINTERNET                3.845343
FAMI_EDUCACIONPADRE               3.347004
FAMI_EDUCACIONMADRE               3.417184
ESTU_PAGOMATRICULAPROPIO          0.938339
RENDIMIENTO_GLOBAL                0.000000
dtype: float64


**Inputar valores nulos para las columnas con valores faltantes**

Se procede a realizar la imputacion utilizando la moda para variables categoricas como:

*   ESTU_VALORMATRICULAUNIVERSIDAD
*   FAMI_ESTRATOVIVIENDA
*   FAMI_TIENEINTERNET
*   FAMI_EDUCACIONPADRE
*   FAMI_EDUCACIONMADRE

Ademas, se procede a imputar con valores representativos de acuerdo al contexto de sus valores unicos, variables categoricas como:
        

*   ESTU_PAGOMATRICULAPROPIO
*   ESTU_HORASSEMANATRABAJA


In [ ]:
# Imputar valores categóricos con la moda (opción más frecuente)
for col in ["ESTU_VALORMATRICULAUNIVERSIDAD", "FAMI_ESTRATOVIVIENDA", "FAMI_TIENEINTERNET",
            "FAMI_EDUCACIONPADRE", "FAMI_EDUCACIONMADRE"]:
    data_cleaned.loc[:, col] = data_cleaned[col].fillna(data_cleaned[col].mode()[0])

# Imputar con valores representativos de acuerdo a su contexto
data_cleaned.loc[:, "ESTU_PAGOMATRICULAPROPIO"] = data_cleaned["ESTU_PAGOMATRICULAPROPIO"].fillna("No")
data_cleaned.loc[:, "ESTU_HORASSEMANATRABAJA"] = data_cleaned["ESTU_HORASSEMANATRABAJA"].fillna("0")

print("Valores faltantes imputados correctamente.")

Valores faltantes imputados correctamente.


**Verificar que no hayan valores nulos**

In [ ]:
# Explorar la cantidad de valores faltantes por columna
missing_values = data_cleaned.isnull().sum()
print("Valores faltantes:\n", missing_values)

Valores faltantes:
 PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


## **Codificacion de variables con orden representativo aplicando OrdinalEnconder**

**Aplicar OrdinalEncoder a la variable objetivo (RENDIMIENTO_GLOBAL) con un orden representativo**

In [ ]:
# Definir el orden correcto de la variable objetivo
rendimiento_order = [['bajo', 'medio-bajo', 'medio-alto', 'alto']]

# Aplicar OrdinalEncoder con el orden correcto definido
ordinal_encoder = OrdinalEncoder(categories=rendimiento_order)
data_cleaned['RENDIMIENTO_GLOBAL'] = ordinal_encoder.fit_transform(data_cleaned[['RENDIMIENTO_GLOBAL']])

# Convertir a entero
data_cleaned['RENDIMIENTO_GLOBAL'] = data_cleaned['RENDIMIENTO_GLOBAL'].astype(int)

# Verificar resultados
print(f"valores unicos: {data_cleaned['RENDIMIENTO_GLOBAL'].unique()}")

valores unicos: [2 0 3 1]


**Aplicar OrdinalEncoder a variables con orden representativo**

In [ ]:
# Crear un diccionario con el orden correcto de las variables categoricas ya inputadas y que tienen un orden logico
ordinal_categories = {
    'ESTU_VALORMATRICULAUNIVERSIDAD': ['No pagó matrícula', 'Menos de 500 mil',
                                       'Entre 500 mil y menos de 1 millón', 'Entre 1 millón y menos de 2.5 millones',
                                       'Entre 2.5 millones y menos de 4 millones', 'Entre 4 millones y menos de 5.5 millones',
                                       'Entre 5.5 millones y menos de 7 millones', 'Más de 7 millones'],
    'ESTU_HORASSEMANATRABAJA': ['0', 'Menos de 10 horas', 'Entre 11 y 20 horas',
                                'Entre 21 y 30 horas', 'Más de 30 horas'],
    'FAMI_ESTRATOVIVIENDA': ['Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3',
                             'Estrato 4', 'Estrato 5', 'Estrato 6']
}

# Aplicar OrdinalEncoder a las variables categoricas
ordinal_enc = OrdinalEncoder(categories=[ordinal_categories[col] for col in ordinal_categories.keys()],
                             handle_unknown='use_encoded_value', unknown_value=-1)

# Transformar las columnas que necesitan codificación ordinal
data_cleaned[list(ordinal_categories.keys())] = ordinal_enc.fit_transform(data_cleaned[list(ordinal_categories.keys())])

#Convertir los resultados de los valores unicos codificados a enteros
data_cleaned[list(ordinal_categories.keys())] = data_cleaned[list(ordinal_categories.keys())].astype(int)

# Verificar los resultados
print(f"valores unicos Valor Matricula: {data_cleaned['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()}")
print(f"valores unicos Horas Semana Trabaja: {data_cleaned['ESTU_HORASSEMANATRABAJA'].unique()}")
print(f"valores unicos Estrato Vivienda: {data_cleaned['FAMI_ESTRATOVIVIENDA'].unique()}")

valores unicos Valor Matricula: [6 4 5 7 3 2 1 0]
valores unicos Horas Semana Trabaja: [1 0 4 3 2]
valores unicos Estrato Vivienda: [3 4 5 2 1 6 0]


**Aplicar OrdinalEnconder a la variable (PERIODO) trandola como con orden representativo**

In [ ]:
# Definir el orden correcto de los periodos academicos
periodo_orden = [[20183, 20184, 20194, 20195, 20196, 20202, 20203, 20212, 20213]]

# Aplicar OrdinalEncoder con el orden correcto definido
ordinal_encoder = OrdinalEncoder(categories=periodo_orden)
data_cleaned["PERIODO"] = ordinal_encoder.fit_transform(data_cleaned[["PERIODO"]])

# Convertir a entero
data_cleaned['PERIODO'] = data_cleaned['PERIODO'].astype(int)

# Verificar transformación
print("Valores únicos de PERIODO transformados:", data_cleaned["PERIODO"].unique())

Valores únicos de PERIODO transformados: [7 6 3 0 2 8 1 5 4]


## **Codificacion de variables booleanas aplicando LabelEnconder**

In [ ]:
# Crear la instancia LabelEnconder
label_encoder = LabelEncoder()

# Aplicar la transformacion a las variables booleanas
data_cleaned['FAMI_TIENEINTERNET'] = label_encoder.fit_transform(data_cleaned['FAMI_TIENEINTERNET'])
data_cleaned['ESTU_PAGOMATRICULAPROPIO'] = label_encoder.fit_transform(data_cleaned['ESTU_PAGOMATRICULAPROPIO'])

#Verificar resultados
print(f"valores unicos Tiene Internet: {data_cleaned['FAMI_TIENEINTERNET'].unique()}")
print(f"valores unicos Pago Matricula Propio: {data_cleaned['ESTU_PAGOMATRICULAPROPIO'].unique()}")

valores unicos Tiene Internet: [1 0]
valores unicos Pago Matricula Propio: [0 1]


#**Pruebas para evaluar la efectividad del modelo empleando XGBoost**

**Carga del preprocesado, division en Train-Test y Codificacion con Target Encoding**

**Nota:** Para las variables de FAMI_EDUCACIONPADRE, FAMI_EDUCACIONMADRE, ESTU_PRGM_DEPARTAMENTO y ESTU_PRGM_ACADEMICO, se decide codificar aplicando Target Encoding y no OneHotEncoder, debido a que tienen muchas categorias, de esta manera se busca reducir de dimensionalidad del modelo sin perder informacion relevante.

Con este metodo se remplaza cada departamento con el promedio del rendimiento global en la respectiva region. De igual manera ocurre con los programas academicos, se remplaza cada programa por el promedio del rendimiento global.

In [ ]:
# Cargar los datos preprocesados
X = data_cleaned.drop(columns=['RENDIMIENTO_GLOBAL'])
y = data_cleaned['RENDIMIENTO_GLOBAL']

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Aplicar Target Encoding solo en Train y transformar Test
target_enc = TargetEncoder()
for col in ["FAMI_EDUCACIONPADRE", "FAMI_EDUCACIONMADRE", "ESTU_PRGM_DEPARTAMENTO", "ESTU_PRGM_ACADEMICO"]:
    X_train[col] = target_enc.fit_transform(X_train[col], y_train)
    X_test[col] = target_enc.transform(X_test[col])  # Se transforma con los valores aprendidos en Train


**Prueba del modelo XGBoost Base**

In [ ]:
# Crear el modelo XGBoost
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Clasificación multiclase
    num_class=len(np.unique(y)),  # Número de clases
    eval_metric='mlogloss',  # Métrica de evaluación
    random_state=42,
    tree_method="hist",
    device="cuda",
    n_jobs=-1
)

# Entrenar el modelo
xgb_model.fit(X_train, y_train)

# Realizar predicciones
y_pred = xgb_model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy del modelo XGBoost:", accuracy)

# Mostrar métricas detalladas
print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred))
print("\nMatriz de Confusión:\n", confusion_matrix(y_test, y_pred))


Accuracy del modelo XGBoost: 0.43767509025270757

Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.46      0.57      0.51     34597
           1       0.34      0.27      0.30     34455
           2       0.33      0.27      0.30     34324
           3       0.56      0.63      0.59     35124

    accuracy                           0.44    138500
   macro avg       0.42      0.44      0.43    138500
weighted avg       0.42      0.44      0.43    138500


Matriz de Confusión:
 [[19697  7502  4796  2602]
 [12477  9374  7649  4955]
 [ 7355  7616  9381  9972]
 [ 2855  3485  6618 22166]]


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:28:07] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
